## Conv2D

In [29]:
import autoreload
%reload_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
sys.path.append('../src')

import tensorflow as tf
from data_processing.mtb_data_provider import MtbDataProvider
from visualization.visualize import MtbVisualizer
from models.conv1d_model import Conv1dModel
from models.conv2d_model import Conv2dModel
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [30]:
RANDOM_SEED = 42

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(RANDOM_SEED)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(RANDOM_SEED)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(RANDOM_SEED)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.set_random_seed(RANDOM_SEED)

# 5. Configure a new global `tensorflow` session
from tensorflow.keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [31]:
def load_data(window_size, raw_files=['neureuth', 'sachrang'], step_size=.25, frequency=40, dismiss_not_riding_label=False):
    X_result = None
    y_result = None
    for raw_file in raw_files:
        X_tmp, y_tmp = MtbDataProvider.load_data('../data/raw/' + raw_file, '*?[0-9].csv', frequency=frequency)
        X_tmp = MtbDataProvider.sync_sensors(X_tmp)
        X_tmp, y_tmp = MtbDataProvider.slice_sensors(X_tmp, y_tmp, 
                                                     window_size=window_size, 
                                                     frequency=frequency, 
                                                     step_size=step_size, 
                                                     dismiss_not_riding_label=dismiss_not_riding_label)

        if X_result is None:
            X_result = X_tmp
            y_result = y_tmp
        else:
            X_result = np.concatenate((X_result, X_tmp))
            y_result = np.concatenate((y_result, y_tmp))

    return X_result, y_result, window_size // frequency

In [34]:
def run_experiments(i=0, epochs=300, batch_size=16, patience=50, frequency=40, dismiss_not_riding_label=False):
    window_sizes = [1000, 2000, 5000, 10000]
    kernel_lengths = [5, 10, 20, 40]
    kernel_height = 2
    
    for window_size in window_sizes:
        X, y, sample_size = load_data(window_size, dismiss_not_riding_label=dismiss_not_riding_label)
        from sklearn.model_selection import train_test_split
        X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=.2, random_state=RANDOM_SEED, shuffle=True)
        
        for kernel_length in kernel_lengths:
            
            if (window_size // frequency < kernel_length):
                continue
            
            model_cnn2 = Conv2dModel.build_model(input_shape=X_train[0].shape, 
                                     cnn_blocks=3,
                                     dropout=0.3,
                                     kernel_size=(kernel_length, kernel_height),
                                     dismiss_not_riding_label=dismiss_not_riding_label)
            
            filepath="../checkpoints/conv2d-" + str(window_size) + "-" + str(kernel_length) + "-{epoch:02d}-{val_sparse_categorical_accuracy:.2f}.hdf5"
            
            checkpoint = ModelCheckpoint(filepath, save_best_only = True, monitor='val_sparse_categorical_accuracy', mode='max')
            early_stopping = EarlyStopping(monitor='val_sparse_categorical_accuracy', min_delta=0, patience=patience, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
            
            class_weight = {
                0: 100.,
                1: 100.,
                2: 100.,
                3: 10.
            }
            
            model_cnn2.fit(X_train, y_train,
              epochs=epochs,
              class_weight=class_weight,
              batch_size=batch_size,
              validation_data=(X_dev, y_dev), 
              callbacks = [checkpoint, early_stopping])
            
            img_base_path = "../checkpoints/img/conv2d-" + str(i) + "-" + str(window_size) + "-" + str(kernel_length) + "-"
            img_path_ending = ".png"

            loss_plot_path = img_base_path  + "accuracy_plot" + img_path_ending
            plt.figure(i)
            plt.plot(model_cnn2.history.history['sparse_categorical_accuracy'])
            plt.plot(model_cnn2.history.history['val_sparse_categorical_accuracy'])
            plt.xlabel('epoch')
            plt.ylabel('sca')
            plt.savefig(loss_plot_path, dpi=300)
            plt.close()
            
            y_pred_dev = model_cnn2.predict(X_dev)
            y_pred_dev = np.argmax(y_pred_dev, axis=1)

            X_dev_plot = X_dev if len(X_dev.shape) <= 3 else X_dev[:,:,0]
            pred_file_path = img_base_path  + "pred_plot" + img_path_ending
            #MtbVisualizer.plot_results(X_dev_plot, y_pred_dev, sample_size, pred_file_path)
            
            act_file_path = img_base_path  + "actual_plot" + img_path_ending
            #MtbVisualizer.plot_results(X_dev_plot, y_dev, sample_size, act_file_path)
            
            confusion_path = img_base_path  + "confusion_matrix" + img_path_ending
            labels = [0,1,2,3] if not dismiss_not_riding_label else [0,1,2]
            print(labels)
            MtbVisualizer.print_confusion_matrix(y_dev, y_pred_dev, labels, confusion_path)


In [5]:
#for i in range(0, 20):
#    run_experiments(i=i, epochs=500, batch_size=32)

In [6]:
#run_experiments(i=0, epochs=500, batch_size=32)

In [ ]:
run_experiments(i=0, epochs=500, batch_size=32, dismiss_not_riding_label=True)

reading  ../data/raw/neureuth_acc_1.csv
reading  ../data/raw/neureuth_acc_2.csv
reading  ../data/raw/neureuth_gyr_1.csv
reading  ../data/raw/neureuth_gyr_2.csv
reading  ../data/raw/sachrang_acc_1.csv
reading  ../data/raw/sachrang_acc_2.csv
reading  ../data/raw/sachrang_gyr_1.csv
reading  ../data/raw/sachrang_gyr_2.csv
Train on 10440 samples, validate on 2611 samples
Epoch 1/500
10440/10440 [==============================] - 3s 321us/sample - loss: 1.4157 - sparse_categorical_accuracy: 0.3748 - val_loss: 1.2402 - val_sparse_categorical_accuracy: 0.4071
Epoch 2/500
10440/10440 [==============================] - 2s 203us/sample - loss: 1.2554 - sparse_categorical_accuracy: 0.3998 - val_loss: 1.2067 - val_sparse_categorical_accuracy: 0.3799
Epoch 3/500
10440/10440 [==============================] - 2s 214us/sample - loss: 1.1737 - sparse_categorical_accuracy: 0.4379 - val_loss: 1.1701 - val_sparse_categorical_accuracy: 0.3872
Epoch 4/500
10440/10440 [==============================] - 2s 21

10440/10440 [==============================] - 2s 207us/sample - loss: 0.9109 - sparse_categorical_accuracy: 0.5739 - val_loss: 0.9396 - val_sparse_categorical_accuracy: 0.5190
Epoch 43/500
10440/10440 [==============================] - 2s 205us/sample - loss: 0.8995 - sparse_categorical_accuracy: 0.5804 - val_loss: 0.9547 - val_sparse_categorical_accuracy: 0.4925
Epoch 44/500
10440/10440 [==============================] - 2s 204us/sample - loss: 0.9027 - sparse_categorical_accuracy: 0.5804 - val_loss: 0.9458 - val_sparse_categorical_accuracy: 0.5067
Epoch 45/500
10440/10440 [==============================] - 2s 214us/sample - loss: 0.9014 - sparse_categorical_accuracy: 0.5862 - val_loss: 0.9540 - val_sparse_categorical_accuracy: 0.4998
Epoch 46/500
10440/10440 [==============================] - 2s 210us/sample - loss: 0.9074 - sparse_categorical_accuracy: 0.5734 - val_loss: 0.9235 - val_sparse_categorical_accuracy: 0.5370
Epoch 47/500
10440/10440 [==============================] - 2s 

10440/10440 [==============================] - 2s 211us/sample - loss: 0.8954 - sparse_categorical_accuracy: 0.5811 - val_loss: 0.9542 - val_sparse_categorical_accuracy: 0.5013
Epoch 86/500
10440/10440 [==============================] - 2s 205us/sample - loss: 0.8968 - sparse_categorical_accuracy: 0.5853 - val_loss: 0.9432 - val_sparse_categorical_accuracy: 0.5182
Epoch 87/500
10440/10440 [==============================] - 2s 212us/sample - loss: 0.8992 - sparse_categorical_accuracy: 0.5787 - val_loss: 0.9411 - val_sparse_categorical_accuracy: 0.5197
Epoch 88/500
10440/10440 [==============================] - 2s 214us/sample - loss: 0.9022 - sparse_categorical_accuracy: 0.5842 - val_loss: 0.9593 - val_sparse_categorical_accuracy: 0.4952
Epoch 89/500
10440/10440 [==============================] - 2s 213us/sample - loss: 0.8949 - sparse_categorical_accuracy: 0.5833 - val_loss: 0.9619 - val_sparse_categorical_accuracy: 0.5013
Epoch 90/500
10440/10440 [==============================] - 2s 

Epoch 38/500
10440/10440 [==============================] - 2s 216us/sample - loss: 0.9212 - sparse_categorical_accuracy: 0.5686 - val_loss: 0.9612 - val_sparse_categorical_accuracy: 0.5155
Epoch 39/500
10440/10440 [==============================] - 2s 215us/sample - loss: 0.9151 - sparse_categorical_accuracy: 0.5754 - val_loss: 0.9440 - val_sparse_categorical_accuracy: 0.5205
Epoch 40/500
10440/10440 [==============================] - 2s 210us/sample - loss: 0.9225 - sparse_categorical_accuracy: 0.5707 - val_loss: 0.9442 - val_sparse_categorical_accuracy: 0.5228
Epoch 41/500
10440/10440 [==============================] - 2s 216us/sample - loss: 0.9248 - sparse_categorical_accuracy: 0.5689 - val_loss: 0.9391 - val_sparse_categorical_accuracy: 0.5308
Epoch 42/500
10440/10440 [==============================] - 2s 212us/sample - loss: 0.9243 - sparse_categorical_accuracy: 0.5724 - val_loss: 0.9413 - val_sparse_categorical_accuracy: 0.5335
Epoch 43/500
10440/10440 [========================

10440/10440 [==============================] - 2s 207us/sample - loss: 0.9069 - sparse_categorical_accuracy: 0.5883 - val_loss: 0.9265 - val_sparse_categorical_accuracy: 0.5454
Epoch 82/500
10440/10440 [==============================] - 2s 209us/sample - loss: 0.9206 - sparse_categorical_accuracy: 0.5821 - val_loss: 0.9288 - val_sparse_categorical_accuracy: 0.5465
Epoch 83/500
10440/10440 [==============================] - 2s 212us/sample - loss: 0.9061 - sparse_categorical_accuracy: 0.5797 - val_loss: 0.9257 - val_sparse_categorical_accuracy: 0.5473
Epoch 84/500
10440/10440 [==============================] - 2s 213us/sample - loss: 0.9083 - sparse_categorical_accuracy: 0.5878 - val_loss: 0.9498 - val_sparse_categorical_accuracy: 0.5331
Epoch 85/500
10440/10440 [==============================] - 2s 213us/sample - loss: 0.9185 - sparse_categorical_accuracy: 0.5815 - val_loss: 0.9396 - val_sparse_categorical_accuracy: 0.5381
Epoch 86/500
10440/10440 [==============================] - 2s 

...

2031/2031 [==============================] - 1s 250us/sample - loss: 0.6941 - sparse_categorical_accuracy: 0.7248 - val_loss: 0.7734 - val_sparse_categorical_accuracy: 0.6772
Epoch 156/500
2031/2031 [==============================] - 0s 241us/sample - loss: 0.6951 - sparse_categorical_accuracy: 0.7287 - val_loss: 0.7464 - val_sparse_categorical_accuracy: 0.7106
Epoch 157/500
2031/2031 [==============================] - 1s 251us/sample - loss: 0.6731 - sparse_categorical_accuracy: 0.7336 - val_loss: 0.8001 - val_sparse_categorical_accuracy: 0.6594
Epoch 158/500
2031/2031 [==============================] - 0s 246us/sample - loss: 0.6752 - sparse_categorical_accuracy: 0.7430 - val_loss: 0.7599 - val_sparse_categorical_accuracy: 0.6831
Epoch 159/500
2031/2031 [==============================] - 0s 245us/sample - loss: 0.6658 - sparse_categorical_accuracy: 0.7336 - val_loss: 0.7958 - val_sparse_categorical_accuracy: 0.6535
Epoch 160/500
2031/2031 [==============================] - 1s 280us/s

2031/2031 [==============================] - 0s 245us/sample - loss: 0.6602 - sparse_categorical_accuracy: 0.7410 - val_loss: 0.8320 - val_sparse_categorical_accuracy: 0.6339
Epoch 199/500
2031/2031 [==============================] - 0s 245us/sample - loss: 0.6772 - sparse_categorical_accuracy: 0.7420 - val_loss: 0.8756 - val_sparse_categorical_accuracy: 0.6260
Epoch 200/500
2031/2031 [==============================] - 1s 258us/sample - loss: 0.6476 - sparse_categorical_accuracy: 0.7602 - val_loss: 0.8470 - val_sparse_categorical_accuracy: 0.6299
Epoch 201/500
2031/2031 [==============================] - 1s 256us/sample - loss: 0.6596 - sparse_categorical_accuracy: 0.7469 - val_loss: 0.8778 - val_sparse_categorical_accuracy: 0.6220
Epoch 202/500
2031/2031 [==============================] - 1s 260us/sample - loss: 0.6413 - sparse_categorical_accuracy: 0.7568 - val_loss: 0.8482 - val_sparse_categorical_accuracy: 0.6378
Epoch 203/500
2031/2031 [==============================] - 1s 251us/s

2031/2031 [==============================] - 0s 230us/sample - loss: 0.8465 - sparse_categorical_accuracy: 0.6509 - val_loss: 1.0238 - val_sparse_categorical_accuracy: 0.5295
Epoch 24/500
2031/2031 [==============================] - 1s 277us/sample - loss: 0.8274 - sparse_categorical_accuracy: 0.6519 - val_loss: 0.9506 - val_sparse_categorical_accuracy: 0.5689
Epoch 25/500
2031/2031 [==============================] - 1s 264us/sample - loss: 0.8316 - sparse_categorical_accuracy: 0.6657 - val_loss: 0.9350 - val_sparse_categorical_accuracy: 0.5709
Epoch 26/500
2031/2031 [==============================] - 0s 234us/sample - loss: 0.8201 - sparse_categorical_accuracy: 0.6755 - val_loss: 0.9862 - val_sparse_categorical_accuracy: 0.5630
Epoch 27/500
2031/2031 [==============================] - 1s 253us/sample - loss: 0.8085 - sparse_categorical_accuracy: 0.6824 - val_loss: 0.9448 - val_sparse_categorical_accuracy: 0.5689
Epoch 28/500
2031/2031 [==============================] - 1s 255us/sample

Epoch 67/500
2031/2031 [==============================] - 1s 278us/sample - loss: 0.7074 - sparse_categorical_accuracy: 0.7341 - val_loss: 0.7233 - val_sparse_categorical_accuracy: 0.7224
Epoch 68/500
2031/2031 [==============================] - 1s 267us/sample - loss: 0.6948 - sparse_categorical_accuracy: 0.7395 - val_loss: 0.6959 - val_sparse_categorical_accuracy: 0.7500
Epoch 69/500
2031/2031 [==============================] - 0s 239us/sample - loss: 0.7026 - sparse_categorical_accuracy: 0.7297 - val_loss: 0.8108 - val_sparse_categorical_accuracy: 0.6358
Epoch 70/500
2031/2031 [==============================] - 1s 261us/sample - loss: 0.7223 - sparse_categorical_accuracy: 0.7189 - val_loss: 0.7663 - val_sparse_categorical_accuracy: 0.6654
Epoch 71/500
2031/2031 [==============================] - 1s 261us/sample - loss: 0.7033 - sparse_categorical_accuracy: 0.7425 - val_loss: 0.7778 - val_sparse_categorical_accuracy: 0.6614
Epoch 72/500
2031/2031 [==============================] - 1s

2031/2031 [==============================] - 1s 252us/sample - loss: 0.6457 - sparse_categorical_accuracy: 0.7573 - val_loss: 0.7290 - val_sparse_categorical_accuracy: 0.7028
Epoch 111/500
2031/2031 [==============================] - 1s 250us/sample - loss: 0.6581 - sparse_categorical_accuracy: 0.7523 - val_loss: 0.6444 - val_sparse_categorical_accuracy: 0.7598
Epoch 112/500
2031/2031 [==============================] - 1s 249us/sample - loss: 0.6722 - sparse_categorical_accuracy: 0.7568 - val_loss: 0.6731 - val_sparse_categorical_accuracy: 0.7421
Epoch 113/500
2031/2031 [==============================] - 1s 252us/sample - loss: 0.6518 - sparse_categorical_accuracy: 0.7632 - val_loss: 0.7371 - val_sparse_categorical_accuracy: 0.6870
Epoch 114/500
2031/2031 [==============================] - 1s 248us/sample - loss: 0.6623 - sparse_categorical_accuracy: 0.7494 - val_loss: 0.7110 - val_sparse_categorical_accuracy: 0.7165
Epoch 115/500
2031/2031 [==============================] - 1s 248us/s

In [27]:
X, y, sample_size = load_data(2000, dismiss_not_riding_label=True)
from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=.2, random_state=RANDOM_SEED, shuffle=True)

reading  ../data/raw/neureuth_acc_1.csv
reading  ../data/raw/neureuth_acc_2.csv
reading  ../data/raw/neureuth_gyr_1.csv
reading  ../data/raw/neureuth_gyr_2.csv
reading  ../data/raw/sachrang_acc_1.csv
reading  ../data/raw/sachrang_acc_2.csv
reading  ../data/raw/sachrang_gyr_1.csv
reading  ../data/raw/sachrang_gyr_2.csv


In [28]:
unique, counts = np.unique(y_train, return_counts=True)
dict(zip(unique, counts))

{0: 1836, 1: 1715, 2: 1675}